<a href="https://colab.research.google.com/github/kristynpantoja/bsg_pytorch/blob/master/BSGcollab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Parameter
import torch.nn.functional as F
import math


from sklearn.datasets import fetch_20newsgroups
import numpy as np
import torch
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

import argparse
from types import SimpleNamespace

# import gensim.downloader as api
# from gensim.models import Word2Vec, FastText, KeyedVectors
# from os.path import isfile

import random
random.seed(1234)

import pandas as pd
import matplotlib.pyplot as plt

import re, string, unicodedata
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer

In [2]:
# arguments 



window = 2
# C = 2*window




In [3]:
# Get corpus: 20 news groups

newsgroups_train = fetch_20newsgroups(subset='train')

# tokenize and preprocess

tokenizer = RegexpTokenizer(u'(?ui)\\b[a-z]{3,}\\b')

tokenized_corpus = [tokenizer.tokenize(document.lower()) for document in newsgroups_train.data]

# normalize_corpus = [normalize(document) for document in tokenized_corpus]

In [4]:
type(tokenizer) == nltk.tokenize.regexp.RegexpTokenizer

True

In [5]:
# get vocabulary

flattened = [word for document in tokenized_corpus for word in document]
vocabulary = set(flattened)
vocabulary = list(vocabulary)
len(vocabulary)

81072

In [ ]:
# # get vocabulary: with some preprocessing
# vectorizer = CountVectorizer(stop_words = 'english', 
#                              min_df=.01, max_df=0.9, 
#                              token_pattern = u'(?ui)\\b[a-z]{3,}\\b')

# count_vecs = vectorizer.fit_transform(newsgroups_train.data)

# # doc_term_matrix = count_vecs.toarray()
# # doc_term_matrix.shape # number of documents, number of words (in vocab)

# # note: vectorizer.get_feature_names() != vectorizer.vocabulary_

# vocabulary = vectorizer.get_feature_names()

In [6]:
# # vocabulary = []
# # for sentence in tokenized_corpus:
# #     for token in sentence:
# #         if token not in vocabulary:
# #             vocabulary.append(token)

word2idx = {w: idx + 1 for (idx , w) in enumerate(vocabulary)}
idx2word = {idx + 1: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

# idx_pairs = []
# # for each sentence
# for sentence in tokenized_corpus:
#     indices = [word2idx[word] for word in sentence]
#     # for each word, threated as center word
#     for center_word_pos in range(len(indices)):
#         # for each window position
#         for w in range(-window_size, window_size + 1):
#             context_word_pos = center_word_pos + w
#             # make soure not jump out sentence
#             if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
#                 continue
#             context_word_idx = indices[context_word_pos]
#             idx_pairs.append((indices[center_word_pos], context_word_idx))

# idx_pairs = np.array(idx_pairs) # it will be useful to have this as numpy array

In [7]:
indexed_corpus = []
# for each document
for document in tokenized_corpus:
    ragged_array = []
    # for each word
    for word in document:
        ragged_array.append(word2idx[word])
    indexed_corpus.append(ragged_array)

In [8]:
print(tokenized_corpus[0])
print(len(tokenized_corpus[0]))
print(indexed_corpus[0])
print(len(indexed_corpus[0]))

['from', 'lerxst', 'wam', 'umd', 'edu', 'where', 'thing', 'subject', 'what', 'car', 'this', 'nntp', 'posting', 'host', 'wam', 'umd', 'edu', 'organization', 'university', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'anyone', 'out', 'there', 'could', 'enlighten', 'this', 'car', 'saw', 'the', 'other', 'day', 'was', 'door', 'sports', 'car', 'looked', 'from', 'the', 'late', 'early', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'the', 'body', 'this', 'all', 'know', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'production', 'where', 'this', 'car', 'made', 'history', 'whatever', 'info', 'you', 'have', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'brought', 'you', 'your', 'neighborhood', 'lerxst']
97
[39987, 69032, 35576, 40308, 3842, 64733, 65043, 30271, 59499, 79491, 17639, 24749, 76239, 27445, 35576, 40308, 3842, 59105, 18639,

In [9]:
padded_corpus = [np.pad(document, (window,window), 'constant', constant_values=(0, 0)) 
                 for document in indexed_corpus]



In [11]:

# test = padded_corpus[0]
# contexts = []
# for context in range(window, len(test) - window):
#     center_word = test[context]
#     context_words = np.concatenate((test[(context - window) : context], 
#                                     test[context + 1 : context + window + 1]))
#     contexts.append((center_word, context_words))
# print(len(contexts))
# print(len(indexed_corpus[0]))
# contexts

In [12]:
# get all the contexts for each word by looking in each document

def document_contexts(document, window):
    contexts = []
    for context in range(window, len(document) - window):
        center_word = document[context]
        context_words = np.concatenate((document[(context - window) : context], 
                                        document[(context + 1) : (context + window + 1)]))
        contexts.append((center_word, context_words))
    return contexts

corpus_document_contexts = [document_contexts(document, window) for document in padded_corpus]




In [13]:
# we just care about contexts right now, so flatten the list
corpus_centers_contexts = [pair for document_contexts in corpus_document_contexts for pair in document_contexts]



In [14]:
corpus_centers_contexts[0:3]

[(39987, array([    0,     0, 69032, 35576])),
 (69032, array([    0, 39987, 35576, 40308])),
 (35576, array([39987, 69032, 40308,  3842]))]

In [15]:
corpus_centers_contexts = list(map(list, zip(*corpus_centers_contexts)))

corpus_center_words = torch.tensor(corpus_centers_contexts[0], dtype = torch.int64)
corpus_context_words = torch.tensor(corpus_centers_contexts[1], dtype = torch.int64)

In [ ]:
corpus_centers_contexts[0:3]

In [ ]:
corpus_center_words[0:3]

In [ ]:
# corpus_center_words_for_negative_sampling

In [16]:
corpus_center_words_for_negative_sampling = corpus_centers_contexts[0]
corpus_centers, corpus_centers_counts = np.unique(corpus_center_words_for_negative_sampling, 
                                                  return_counts=True)
unigram_dict = dict(zip(corpus_centers, corpus_centers_counts/len(corpus_center_words)))


In [ ]:
corpus_centers_contexts[0]

In [ ]:
corpus_center_words.tolist()

In [32]:
class BSG(nn.Module):
    def __init__(self, unigram_dict, vocab_size, input_dim=50, hidden_dim=50, latent_dim=100, margin=1., model_name='BSG with the hinge loss'):
        super().__init__()
        """
        :param vocab_size: the number of unique words
        :param input_dim: the number of components in the encoder's word embeddings
        :param hidden_dim: the number of components in the encoder's hidden layer
        :param latent_dim: the number of components in the latent vector(also output word mu's)
        :param margin: margin constant present in the hinge loss
        """
        self.window = window
        self.model_name = model_name
        self.vocab_size = vocab_size
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.latent_dim = latent_dim
        self.margin = margin
        
        self.unigram_dist = torch.distributions.Categorical(torch.tensor(list(unigram_dict.values())))

        # assign full parameters
#         self.params_full = self.__build_model()

        # extract only the actual parameter data-structures(tensors) as those will be optimized
#         self.params = [param.value for param in self.params_full.values()]

        # user accessible functions build ( e.g. training functions)
#         self.__build_functions()

        # encoder layers
        self.encoder_embedding = nn.Embedding(self.vocab_size+1, self.input_dim, padding_idx = 0) # R
        self.encoder_lin1 = nn.Linear(self.input_dim*2, self.hidden_dim) # M
        self.encoder_mu = nn.Linear(self.hidden_dim, self.latent_dim) # U -> mu
        self.encoder_logsigma = nn.Linear(self.hidden_dim, 1) # W -> log sigma
        
        # word embeddings' parameters for normal distributions of word types
        self.type_means = nn.Embedding(self.vocab_size+1, self.latent_dim)
        self.type_logvars = nn.Embedding(self.vocab_size+1, 1)
        
        
    def encoder(self, centers_batch, contexts_batch):
#             batch_size = centers_batch.shape[0]
        b, C = contexts_batch.shape
        assert C == 2*self.window, "C does not equal 2*window"
        embed_centers = self.encoder_embedding(centers_batch)
        centers_with_3rd_dim = embed_centers.unsqueeze(1) # batch by 1 by hidden
        repr_center = centers_with_3rd_dim.repeat(1, C, 1) # centers as a matrix
        repr_context = self.encoder_embedding(contexts_batch)
        
        repr_common = torch.cat((repr_center, repr_context), 2)
        
        hidden = F.relu(self.encoder_lin1(repr_common)).sum(1) # ?
        mu = self.encoder_mu(hidden)
        logsigma = self.encoder_logsigma(hidden)
        return mu, logsigma
        
#         sums = []
#         for center, context in zip(centers_batch, contexts_batch):
#             embed_center = self.encoder_embedding(center)
#             embed_context = self.encoder_embedding(context)
#             assert embed_context.shape[1] == self.input_dim, "context embedding is not a 2d tensor"
#             center_repeats = embed_center.repeat(2*window, 1)
#             concat = torch.cat((embed_context, center_repeats),1)
#             sum_relu_en1 = F.relu(self.encoder_lin1(concat)).sum(0) # a vector
#             sums.append(sum_relu_en1) # the vectors of sums
#         sums = torch.stack(sums)
#         mu = self.encoder_mu(sums)
#         logsigma = self.encoder_logsigma(sums)
#         return mu, logsigma
    
    def reparameterize(self, centers_batch, posterior_mean, posterior_logvar):
        eps = Variable(centers_batch.data.new().resize_as_(posterior_mean.data).normal_())
        z = posterior_mean + posterior_logvar.exp().sqrt() * eps
        return z
    
    def KL(self, word_idx, post_mu, post_logsigma):
        post_sigma = post_logsigma.exp()
        type_mean = self.type_means(word_idx)
        type_var = self.type_logvars(word_idx).exp().view([-1,post_logsigma.shape[1]])
#         print("type_var")
#         print(type_var.shape)
        var_division = post_sigma / type_var
        diff = post_mu - type_mean
        diff_term = (diff * diff).sum(1) / type_var ## added the .sum(1) from the original KL in our AVITM implementation of KL
        logvar_division = type_var.log() - post_logsigma
        # compute KL
        KLD = 0.5 * ( (var_division + diff_term + logvar_division).sum(1) - self.latent_dim )
        return KLD
        
    def forward(self, centers_batch, contexts_batch):
#         print("contexts_batch.shape - should be batch size by context window size")
#         print(contexts_batch.shape)
        mu, logsigma = model.encoder(centers_batch, contexts_batch)
        # repeat mu, logsigma 2*window times, 
        #  one for each context word in an input
        #  - do this for each input in the batch
        mus = mu.repeat(1,window*2).view(-1,mu.shape[1]) 
        logsigmas = logsigma.repeat(1,window*2).view(-1,logsigma.shape[1])
        # compute KLs
        KL_contexts = self.KL(contexts_batch.view([-1,1]), mus, logsigmas)
        negative_contexts_batch = self.unigram_dist.sample(contexts_batch.shape) + 1
        KL_negative_contexts = self.KL(negative_contexts_batch.view([-1,1]), mus, logsigmas)
        KL_center_word = self.KL(centers_batch, mu, logsigma)
        
        # compute hard margin of KLs of negative and positive context words
        hard_margin_arg = KL_contexts - KL_negative_contexts + self.margin
        loss = torch.max(hard_margin_arg, torch.zeros_like(hard_margin_arg)).sum() + KL_center_word.sum()
        return loss / centers_batch.shape[0]
    
# b, window_size = pos_context_words.shape
#         sigma_q = T.repeat(sigma_q, window_size, axis=0)
#         mu_q = T.repeat(mu_q, window_size, axis=0)

#         pos_c_resh = pos_context_words.reshape((-1, ))
#         mu_p_pos, sigma_p_pos = self.__compute_prior_params(pos_c_resh)

#         neg_c_resh = neg_context_words.reshape((-1, ))
#         mu_p_neg, sigma_p_neg = self.__compute_prior_params(neg_c_resh)

#         kl_pos = self.kl(mu_q, sigma_q, mu_p_pos, sigma_p_pos).reshape((b, -1))
#         kl_neg = self.kl(mu_q, sigma_q, mu_p_neg, sigma_p_neg).reshape((b, -1))

#         # hard margin
#         return T.sum(T.maximum(0.0, self.margin - kl_neg + kl_pos) * mask, axis=1)
    
def train(model, args, optimizer, center_words, context_words):
    '''
    model - object of class BSG
    args - dict of args
    optimizer - nn.optim
    centers_batch, contexts_batch
    '''
    for epoch in range(args.num_epoch):
        all_indices = torch.randperm(context_words.size(0)).split(args.batch_size)
        loss_epoch = 0.0
        model.train()                   # switch to training mode
        for batch_indices in all_indices:
            if not args.nogpu: batch_indices = batch_indices.cuda()
            context_words_input = Variable(context_words[batch_indices])
            center_words_input = Variable(center_words[batch_indices])
            loss = model(center_words_input, context_words_input)
            # optimize
            optimizer.zero_grad()       # clear previous gradients
            loss.backward()             # backprop
            optimizer.step()            # update parameters
            # report
            loss_epoch += loss.data[0]    # add loss to loss_epoch
        if epoch % 5 == 0:
            print('Epoch {}, loss={}'.format(epoch, loss_epoch / len(all_indices)))
#         model.losses.append(loss_epoch / len(all_indices))

    return model
    

In [33]:
# arguments for BSG model

args_dict = {"vocab_size" : vocabulary_size, "window" : window, 
             "batch_size" : 200, "optimizer" : 80, "learning_rate" : 0.0005, 
             "momentum" : 0.99, "num_epoch" : 50, "init_mult" : 1, 
             "variance" : 0.995, "start" : True, "nogpu" : True, 
             "embedding_dim" : 300, "freeze" : False}

args = SimpleNamespace(**args_dict)
# args.num_input = doc_term_matrix_tensor.shape[1]


In [34]:
model = BSG(unigram_dict, args.vocab_size)

In [35]:
mu, logsigma = model.encoder(corpus_center_words[0:2], corpus_context_words[0:2])

In [ ]:
loss = model.forward(corpus_center_words[0:2], corpus_context_words[0:2]) + 1

In [ ]:
loss

In [ ]:
corpus_center_words[0:2]

In [ ]:
KLs.shape

In [ ]:
##### repeat mus in the right order

# mus = []
# for row in mu:
#     mus.append([row.repeat(window, 1)])
# flattened = [word for document in mus for word in document]
# torch.cat(flattened)

In [ ]:
corpus_context_words[0:2].view([-1,1]).shape

In [ ]:
#corpus_context_words[0:2].reshape((-1,))

In [ ]:
mu

In [ ]:
mu.repeat(1,window*2).view(-1,mu.shape[1])

In [36]:
optimizer = torch.optim.Adam(model.parameters(), args.learning_rate, betas=(args.momentum, 0.999))



In [37]:
model = train(model, args, optimizer, corpus_center_words[0:1000], corpus_context_words[0:1000])



/anaconda3/envs/math689env/lib/python3.7/site-packages/ipykernel/__main__.py:147: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Epoch 0, loss=271148.875
Epoch 5, loss=182078.59375
Epoch 10, loss=161547.96875


KeyboardInterrupt: 